In [85]:
import ctypes, inspect, os, graphlab
from ctypes import wintypes
kernel32 = ctypes.WinDLL('kernel32', use_last_error=True)
kernel32.SetDllDirectoryW.argtypes = (wintypes.LPCWSTR,)
src_dir = os.path.split(inspect.getfile(graphlab))[0]
kernel32.SetDllDirectoryW(src_dir)

1

In [86]:
import graphlab

In [87]:
products = graphlab.SFrame('amazon_baby.gl/')

In [88]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [89]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [90]:
#define words of interest

In [91]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [92]:
#define a python method that SArray.apply takes tp transform existing entry

In [93]:
def awesome_count(word_count_column_entry, word):
    a = 0
    if word in word_count_column_entry:
        a += word_count_column_entry[word]
    return a

In [94]:
#for each interesting words of selection, create a new column named <word>

In [95]:
for word in selected_words:
    products[word] = products['word_count'].apply(lambda x:awesome_count(x,word))

In [96]:
#calculate the sum of each column and output as dict

In [97]:
p = {}
for word in selected_words:
    p[word] = products[word].sum()
print p

{'fantastic': 932L, 'love': 42065L, 'bad': 3724L, 'awesome': 2090L, 'great': 45206L, 'terrible': 748L, 'amazing': 1363L, 'horrible': 734L, 'awful': 383L, 'hate': 1220L, 'wow': 144L}


In [98]:
#positive sentiment = 4* or 5* reviews
#ignore all 3* reviews
products = products[products['rating'] != 3]
products['sentiment'] = products['rating'] >=4

In [99]:
#Create a new sentiment analysis model using only the selected_words as features

In [100]:
train_data,test_data = products.random_split(.8, seed=0)

In [101]:
print train_data[0]['review']

it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.


In [102]:
#Train a logistic regression classifier (use graphlab.logistic_classifier.create) using just the selected_words. 
#Hint: you can use this parameter in the .create() call 
#to specify the features used to be exactly the new columns you just created:

In [103]:
selected_words_model = graphlab.logistic_classifier.create(train_data, target="sentiment", features=selected_words,validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.295774     | 0.844269          | 0.843142            |

| 5         | 6        | 0.357939     | 0.844269          | 0.843142            |

| 6         | 7        | 0.433138     | 0.844269          | 0.843142            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [104]:
selected_words_model['coefficients']

name,index,class,value,stderr
(intercept),None,1,1.36728315229,0.00861805467824
awesome,None,1,1.05800888878,0.110865296265
great,None,1,0.883937894898,0.0217379527921
fantastic,None,1,0.891303090304,0.154532343591
amazing,None,1,0.892802422508,0.127989503231
love,None,1,1.39989834302,0.0287147460124
horrible,None,1,-1.99651800559,0.0973584169028
bad,None,1,-0.985827369929,0.0433603009142
terrible,None,1,-2.09049998487,0.0967241912229
awful,None,1,-1.76469955631,0.134679803365


In [ ]:
#calculate selected_words model, accuracy on test data is ~0.843

In [105]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8431419649291376,
 'auc': 0.6648096413721418,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      1       |        0        |  130  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.914242563530107,
 'log_loss': 0.4054747110366022,
 'precision': 0.8453551912568306,
 'recall': 0.9953531598513011,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   | 

In [106]:
diaper_champ_reviews = products[products['name']=='Baby Trend Diaper Champ']

In [107]:
diaper_champ_reviews.head()

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'just': 2L, 'less': 1L,'-': 3L, 'smell- ...",0,0,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'just': 1L, 'less': 1L,'when': 3L, 'over': 1L, ...",0,0,0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'control': 1L, 'am': 1L,'it': 1L, 'used': 1L, ...",0,0,0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'and': 3L, 'over.': 1L,'all': 1L, 'love': 1L, ...",0,0,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1L, 'when': 1L,'both': 1L, 'results': ...",0,0,0
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'lysol': 1L, 'all': 1L,'mom.': 1L, 'busy': 1L, ...",0,0,0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'all': 1L, 'bags.': 1L,'just': 1L, ""don't"": 2L, ...",0,0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1L, 'all': 2L,'bags.': 1L, 'feedback': ...",0,0,0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'and': 2L, 'all': 1L,'just': 1L, 'is': 2L, ...",0,0,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1L,'(required': 1L, ...",0,0,0


In [108]:
diaper_champ_reviews['predicted_sentiment'] = selected_words_model.predict(diaper_champ_reviews, output_type='probability')

In [115]:
sentiment_model = graphlab.logistic_classifier.create(train_data, target="sentiment", features=['word_count'],validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 6         | 13       | 3.000000  | 3.384979     | 0.899991          | 0.825967            |

| 10        | 18       | 1.000000  | 5.176735     | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [ ]:
#calculate all_words model, accuracy on test data is ~0.916

In [116]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.916256305548883,
 'auc': 0.9446492867438502,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  1461 |
 |      0       |        1        |  1328 |
 |      0       |        0        |  4000 |
 |      1       |        1        | 26515 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9500349343413533,
 'log_loss': 0.2610669843242166,
 'precision': 0.9523039902309378,
 'recall': 0.9477766657134686,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |  

In [ ]:
#calculate the accuracy of majority classifier ~0.84

In [142]:
float(len(products[products['sentiment'] == 1]))/float(len(products))

0.8411233448474381

In [ ]:
#find the most positive review predicted by "all words model"

In [138]:
diaper_champ_reviews['predicted_sentiment_all_words'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [141]:
all_word_model_prediction = diaper_champ_reviews.sort('predicted_sentiment_all_words')

In [ ]:
#this entry contains both model's result in 2 columns
#selected words model is junk because many reviews don't have enough selected words, thus contributing little to the model

In [ ]:
all_word_model_prediction[-1]